[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_tensorrt_diffusers.ipynb)

In [ ]:
%cd /content
!git clone -b release/9.1 --recursive https://github.com/NVIDIA/TensorRT
%cd /content/TensorRT

%cd /content
!wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/secure/9.1.0/tars/tensorrt-9.1.0.4.linux.x86_64-gnu.cuda-11.8.tar.gz
!mkdir trt
%cd /content/trt
!tar -xvzf /content/tensorrt-9.1.0.4.linux.x86_64-gnu.cuda-11.8.tar.gz

%cd /content/TensorRT
!mkdir -p /content/TensorRT/build
%cd /content/TensorRT/build
!cmake .. -DTRT_LIB_DIR=/content/trt/TensorRT-9.1.0.4/lib -DTRT_OUT_DIR=/content/TensorRT/build/out -DCUDNN_ROOT_DIR=/usr/local/cuda/bin/nvcc -DGPU_ARCHS="75"
!make -j$(nproc)

In [ ]:
# https://github.com/NVIDIA/TensorRT/blob/release/8.6/demo/experimental/HuggingFace-Diffusers/TensorRT-diffusers-txt2img.ipynb
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-text2image-stable-diffusion-pipeline
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-image2image-stable-diffusion-pipeline
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-inpainting-stable-diffusion-pipeline

# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_txt2img.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_img2img.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_inpaint.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/run_tensorrt_controlnet.py

!pip install -q tensorrt diffusers transformers accelerate
!pip install -q --extra-index-url https://pypi.ngc.nvidia.com "onnx-graphsurgeon" "onnxruntime" "polygraphy"

In [ ]:
import torch
from diffusers.pipelines.stable_diffusion import StableDiffusionPipeline
from diffusers import DDIMScheduler

scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="scheduler")
pipe_trt = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", custom_pipeline="stable_diffusion_tensorrt_txt2img", revision='fp16',
    torch_dtype=torch.float16, scheduler=scheduler, image_height=512, image_width=512)

pipe_trt.set_cached_folder("stabilityai/stable-diffusion-2-1", revision='fp16')

pipe_trt = pipe_trt.to("cuda")

In [ ]:
prompt = "a beautiful photograph of Mt. Fuji during cherry blossom"

# warm up runs to stabilize performance benchmarking
num_warm_up_steps=5
for _ in range(num_warm_up_steps):
    _ = pipe_trt(prompt)

image = pipe_trt(prompt).images[0]
display(image)